In [1]:
import pandas as pd
import numpy as np

In [2]:
# Reading data in
data = pd.read_csv('raw_data.csv')

In [3]:
# Defining function to calculate emas
def calculate_ema(closes, n):
    # Fromula for alpha
    alpha = 2/(n+1)

    # Initializing variables
    ema = np.zeros(len(closes))
    ema[0] = closes.iloc[0]

    # Looping through all closing prices to calculate ema
    for i in range(1, len(data)):
        ema[i] = (alpha * closes.iloc[i]) + ((1-alpha) * ema[i-1])
    
    return ema

# Defining function to calculate ATR
def calculate_ATR(high, low, close, n):
    # Initializng variables
    TR = np.zeros(len(high))
    ATR = np.zeros(len(high))

    # Setting value for yesterday close
    y_close = close.shift(1)

    # Taking values for calculation of true range(TR)
    H_L = high - low
    H_y = abs(high - y_close)
    L_y = abs(low - y_close)

    # Calculating TR
    for i in range(0, len(high)):
        TR[i] = max(H_L.iloc[i], H_y.iloc[i], L_y.iloc[i])
    
    # Initializing ATR
    ATR[n] = (1 / n) * sum(TR[:n])

    # Calcualting ATR
    for i in range(n+1, len(high)):
        ATR[i] = ((ATR[i-1] * (n-1)) + TR[i]) / n

    return ATR

In [4]:
# Creating 5, 10, 20 period emas
for n in [5,10,20]:
    data[f'ema_{n}'] = calculate_ema(data['close'], n)

# Adding atr column
data['atr'] = calculate_ATR(data['high'], data['low'], data['close'], 20)

# Adding KC bands
data['upper_band'] = data['ema_20'] + 4 * data['atr']
data['lower_band'] = data['ema_20'] - 4 * data['atr']

In [5]:
# Removing 0.0 atr values
data = data.tail(data.shape[0] - 20)
data = data.copy()

In [6]:
# Saving to csv
data.to_csv('technicals.csv')